##extra functionality for optimization

In [ ]:
#add notr range manualy 
def add_note_range_column(songs_dict, expert_data):
    note_ranges = [songs_dict.get(filename) for filename in expert_data['midi_filename']]
    expert_data['note_range'] = note_ranges
    pass
import json
#for adding note range
# Load dictionary from JSON file
with open('song_dict.json', 'r') as f:
    song_dict = json.load(f)
add_note_range_column(song_dict, expert_data)

In [ ]:
def loss(sum_diff, length):
        #start with avg diff, need to define loss
        return sum_diff / length

In [ ]:
def diff_function(prediction, expert):
    #get two vectors containing orederd predictions and expert data practice modes
    diff = 0
    for i, pred in enumerate(prediction):
        if pred != expert[i]:
            diff+=1
    return loss(diff, len(prediction))
    pass  

In [ ]:
def policy_diff(gauss_model, recorded_points, expert_practice_mode):
         #calculate difference between the prediction of the gauss model and the expert advice.
      r_p =  recorded_points["error_before_right_timing", "error_before_right_pitch"]
      #a question- the diff should be calculated binaricly(by prediction) or by the utility diffs?
      policy_diff =diff_function(gauss_model.get_best_practice_mode(r_p),  expert_practice_mode)
      return policy_diff
      pass


In [ ]:
def policy_diff_utility(gauss_model, point, expert_opt_policy):
    
      c=0#what is c?
      policy_diff= compare_to_best_policy(gauss_model.get_policy(c),point["utility"])
      return policy_diff

In [ ]:
def initialize_gp_generelize():
    #generelized to any number of kernels, not sure if possible
    gp_models = []
    kernel_hyper_params = [{'lengthscale': 1.0}, {'lengthscale': 0.5}, {'lengthscale': 2.0}]
    kernels = [GPy.kern.Matern52, GPy.kern.Matern32, GPy.kern.RBF]

    for kernel, hyper_params in zip(kernels, kernel_hyper_params):
        for param in hyper_params:
            # initialize a new kernel with the given hyperparameters
            kernel_instance = kernel(input_dim=1, **param)

            # initialize a new Gaussian process model with the current kernel
            gp = GaussianProcess(kernel=kernel_instance)

            gp_models.append(gp)

    return gp_models
pass